In [1]:
import fetch_data as fd
import svm
from random import shuffle
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
import kernel
import numpy as np
from sklearn import metrics
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import time
import subprocess
import io
import sys
import alignment
import os

In [2]:
def parseMatrix(matrix):
    num_rows = int(matrix[0])
    num_cols = int(matrix[1])
    K = np.zeros([num_rows,num_cols])
    counter = 2
    for i in range(num_rows):
        for j in range(num_cols):
            K[i,j] = float(matrix[counter])
            counter = counter + 1
    return K

def marcus_recursive_kernel(s,t,n,l):
    the_strings = b''
    the_args = bytes(str(len(s)),'ascii')+ b' ' + bytes(str(len(t)),'ascii')+ b' '
    for i in s:
        the_strings = the_strings + bytes(i,'ascii')
        the_args = the_args + bytes(str(len(i)),'ascii') + b' '
    for i in t:
        the_strings = the_strings + bytes(i,'ascii')
        the_args = the_args + bytes(str(len(i)),'ascii') + b' '
    the_args = the_args + bytes(str(n), 'ascii')+ b' ' + bytes(str(l), 'ascii')
    fast_kernel = subprocess.Popen(["./fast_recursive_kernel.out"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    output_test = fast_kernel.communicate(input=b'1 '+the_args+the_strings)[0]
    output_test_list = output_test.decode('utf-8').split()
    K = parseMatrix(output_test_list)
    return K

def create_ngrams(text, n):
    """Create a set of ngrams of length n"""
    return set(text[i:i+n] for i in range(len(text)-n+1))

def wk(doc1, doc2):
    #print "Creating the bag of words...\n"
    clean_docs = [doc1,doc2]
    #defaultanalyzer "word" removes non-chars in preprocessing and tokenizes words. does not remove "markup tokens"
    #stop_words should be "english" if not using clean_input_docs()
    #vectorizer = CountVectorizer(analyzer = "word",   \
    #                             tokenizer = None,    \
    #                             preprocessor = None, \
    #                             stop_words = "english") 
    vectorizer = CountVecotizer(analyzer)

    train_data_features = vectorizer.fit_transform(clean_docs)
    train_data_features = train_data_features.toarray()

    transformer = TfidfTransformer(smooth_idf=False)
    tfidf = transformer.fit_transform(train_data_features)
    tfidf = tfidf.toarray() 
    return np.dot(tfidf[0],tfidf[1])


def ngk(doc1, doc2, n,luc=False):
    if(luc):
        p1 = [doc1[i:i+n] for i in range(len(doc1)-n+1)]
        p2 = [doc2[i:i+n] for i in range(len(doc2)-n+1)]
        same = 0;
        unique = 0;
        same = len(np.intersect1d(p1,p2))
        unique = same+len(np.setdiff1d(p1,p2))+len(np.setdiff1d(p2,p1));
        return same/unique
    else:
        sd1 = create_ngrams(doc1, n)
        sd2 = create_ngrams(doc2, n)

        if len(sd1 | sd2) == 0:
            return 1.0

        return len(sd1 & sd2) * 1.0 / len(sd1 | sd2)

    

def testNGK(train,trainLabels,test,testLabels,n=2,luc=False):
    
    clf = svm.SVC(kernel='precomputed')
    
    Ktrain = np.zeros([len(train),len(train)]);
    #computeing train
    print("beginning to train")
    for i in tqdm(range(len(train))):
        for j in range(len(train)):
            Ktrain[i][j] = ngk(train[i],train[j],n,luc);
    
    Ktest = np.zeros([len(train),len(test)]);
    print("done training. \n starting to predict")
    for i in tqdm(range(len(train))):
        for j in range(len(test)):
            Ktest[i][j] = ngk(train[i],test[j],n,luc)
            
    clf.fit(Ktrain,trainLabels)
    
    pred = clf.predict(Ktest.T)
    
    prec, rec, fsc, supp = metrics.precision_recall_fscore_support(testLabels, pred)
    
    return prec, rec, fsc, supp
    
    

In [52]:
#generate data

categories = ['earn','acq','crude','corn']
numberOfTraining = [152,114,76,38]
numberOfTesting = [40,25,15,10]

trainData,trainLabels, testData,testLabel = fd.loadData(categories,numberOfTraining,numberOfTesting)

minTrain = 100;
for i in trainData:
    if(len(i)<minTrain):
        minTrain = len(i)
        
minTest = 100;
for i in testData:
    if(len(i)<minTest):
        minTest = len(i)
        
print(minTrain)
print(minTest)

62
63


In [53]:
#save all data 

cwd = os.getcwd()+'\..\data\clean_data';
kerns = os.path.join(cwd,'trainData_8.npy')
np.save(kerns, trainData) 

cwd = os.getcwd()+'\..\data\clean_data';
kerns = os.path.join(cwd,'testData_8.npy')
np.save(kerns, testData) 

cwd = os.getcwd()+'\..\data\clean_data';
kerns = os.path.join(cwd,'trainLabels_8.npy')
np.save(kerns, trainLabels) 

cwd = os.getcwd()+'\..\data\clean_data';
kerns = os.path.join(cwd,'testLabels_8.npy')
np.save(kerns, testLabel) 


In [42]:
#load data
cwd = os.getcwd()+'\..\data\clean_data';
kerns = os.path.join(cwd,'trainData.npy')
trainData = np.load(kerns)

kerns = os.path.join(cwd,'testData.npy')
testData = np.load(kerns)

kerns = os.path.join(cwd,'trainLabels.npy')
trainLabels = np.load(kerns)

kerns = os.path.join(cwd,'testLabels.npy')
testLabels = np.load(kerns)


In [43]:
n=4
l=0.5

t1=time.time()
K = marcus_recursive_kernel(trainData,trainData,n,l)
t2=time.time()

print("marcus: "+repr(t2-t1))

marcus: 1331.7753159999847


In [44]:
cwd = os.getcwd()+'\..\data\precomp_recursive_kernels';
kerns = os.path.join(cwd,'n4l05Train.npy')
np.save(kerns,K)


In [32]:
cwd = os.getcwd()+'\..\data\precomp_recursive_kernels';
kerns = os.path.join(cwd,'n3l05Train.npy')
Ktrain = np.load(kerns)

kerns = os.path.join(cwd,'n3l05Test.npy')
Ktest = np.load(kerns)

In [40]:
earnTrain = np.array([0]*152)
acqTrain = np.array([1]*114)
crudeTrain = [2]*76
cornTrain = [3]*38
trainLbl = np.concatenate([earnTrain,acqTrain,crudeTrain,cornTrain])

earnTest = np.array([0]*40)
acqTest = np.array([1]*25)
crudeTest = [2]*15
cornTest = [3]*10
testLbl = np.concatenate([earnTest,acqTest,crudeTest,cornTest])

clf = svm.SVC(kernel='precomputed');

clf.fit(Ktrain,trainLbl);

pred = clf.predict(Ktest)

print(pred)

prec, rec, fsc, supp = metrics.precision_recall_fscore_support(testLbl, pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 3 3 3 2 3 3 2 3]


In [41]:
print(prec)
print(rec)
print(fsc)

[ 0.97297297  0.82758621  0.77777778  1.        ]
[ 0.9         0.96        0.93333333  0.6       ]
[ 0.93506494  0.88888889  0.84848485  0.75      ]
